# Multi-spot Gamma Fitting

In [ ]:
from fretbursts import fretmath

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from cycler import cycler
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format='retina'  # for hi-dpi displays

In [ ]:
import matplotlib as mpl
from cycler import cycler

bmap = sns.color_palette("Set1", 9)
colors = np.array(bmap)[(1,0,2,3,4,8,6,7), :]
mpl.rcParams['axes.prop_cycle'] = cycler('color', colors)
colors_labels = ['blue', 'red', 'green', 'violet', 'orange', 'gray', 'brown', 'pink', ]
for c, cl in zip(colors, colors_labels):
    locals()[cl] = tuple(c) # assign variables with color names
sns.palplot(colors)

In [ ]:
sns.set_style('whitegrid')

# Load Data

## Multispot

Load the **leakage coefficient** from disk (computed in [Multi-spot 5-Samples analyis - Leakage coefficient fit](Multi-spot 5-Samples analyis - Leakage coefficient fit.ipynb)):

In [ ]:
leakage_coeff_fname = 'results/Multi-spot - leakage coefficient KDE wmean DexDem.csv'
leakageM = float(np.loadtxt(leakage_coeff_fname, ndmin=1))

print('Multispot Leakage Coefficient:', leakageM)

Load the **direct excitation coefficient** ($d_{dirT}$) from disk (computed in [usALEX - Corrections - Direct excitation physical parameter](usALEX - Corrections - Direct excitation physical parameter.ipynb)):

In [ ]:
dir_ex_coeff_fname = 'results/usALEX - direct excitation coefficient dir_ex_t beta.csv'
dir_ex_t = float(np.loadtxt(dir_ex_coeff_fname, ndmin=1))

print('Direct excitation coefficient (dir_ex_t):', dir_ex_t)

Multispot PR for FRET population:

In [ ]:
mspot_filename = 'results/Multi-spot - dsDNA - PR - all_samples all_ch.csv'

E_pr_fret = pd.read_csv(mspot_filename, index_col=0)
E_pr_fret

## usALEX

Corrected $E$ from μs-ALEX data:

In [ ]:
data_file = 'results/usALEX-5samples-E-corrected-all-ph.csv'
data_alex = pd.read_csv(data_file).set_index('sample')#[['E_pr_fret_kde']]
data_alex.round(6)

In [ ]:
E_alex = data_alex.E_gauss_w
E_alex

# Multi-spot gamma fitting

In [ ]:
import lmfit

In [ ]:
def residuals(params, E_raw, E_ref):
    gamma = params['gamma'].value
    # NOTE: leakageM and dir_ex_t are globals
    return E_ref - fretmath.correct_E_gamma_leak_dir(E_raw, leakage=leakageM, gamma=gamma, dir_ex_t=dir_ex_t)

In [ ]:
params = lmfit.Parameters()
params.add('gamma', value=0.5) 

In [ ]:
E_pr_fret_mean = E_pr_fret.mean(1)
E_pr_fret_mean

In [ ]:
m = lmfit.minimize(residuals, params, args=(E_pr_fret_mean, E_alex))
lmfit.report_fit(m.params, show_correl=False)

In [ ]:
E_alex['12d'], E_pr_fret_mean['12d']

In [ ]:
m = lmfit.minimize(residuals, params, args=(np.array([E_pr_fret_mean['12d']]), np.array([E_alex['12d']])))
lmfit.report_fit(m.params, show_correl=False)

In [ ]:
print('Fitted gamma(multispot):', m.params['gamma'].value)

In [ ]:
multispot_gamma = m.params['gamma'].value
multispot_gamma

In [ ]:
E_fret_mch = fretmath.correct_E_gamma_leak_dir(E_pr_fret, leakage=leakageM, dir_ex_t=dir_ex_t, 
                                               gamma=multispot_gamma)
E_fret_mch = E_fret_mch.round(6)
E_fret_mch

In [ ]:
E_fret_mch.to_csv('results/Multi-spot - dsDNA - Corrected E - all_samples all_ch.csv')

In [ ]:
'%.5f' % multispot_gamma

In [ ]:
with open('results/Multi-spot - gamma factor.csv', 'wt') as f:
    f.write('%.5f' % multispot_gamma)

In [ ]:
norm = (E_fret_mch.T - E_fret_mch.mean(1))#/E_pr_fret.mean(1)
norm_rel = (E_fret_mch.T - E_fret_mch.mean(1))/E_fret_mch.mean(1)
norm.plot()
norm_rel.plot()

### Plot FRET vs distance

In [ ]:
sns.set_style('whitegrid')

In [ ]:
CH = np.arange(8)
CH_labels = ['CH%d' % i for i in CH]
dist_s_bp = [7, 12, 17, 22, 27]

In [ ]:
fontsize = 16

fig, ax = plt.subplots(figsize=(8, 5))

ax.plot(dist_s_bp, E_fret_mch, '+', lw=2, mew=1.2, ms=10, zorder=4)
ax.plot(dist_s_bp, E_alex, '-', lw=3, mew=0, alpha=0.5, color='k', zorder=3)

plt.title('Multi-spot smFRET dsDNA, Gamma = %.2f' % multispot_gamma)
plt.xlabel('Distance in base-pairs', fontsize=fontsize); 
plt.ylabel('E', fontsize=fontsize)
plt.ylim(0, 1); plt.xlim(0, 30)
plt.grid(True)
plt.legend(['CH1','CH2','CH3','CH4','CH5','CH6','CH7','CH8', u'μsALEX'], 
       fancybox=True, prop={'size':fontsize-1},
       loc='best');

> **NOTE** The fact the we fit the 27d with a single Gaussian may account for the slight shift of the FRET efficiency compared to the us-ALEX measurements. The shift is bigger when using an asymmetric-gaussian model for multi-spot fitting. Probably, for consistency with us-ALEX fitting we should stick to the plain gaussian.